In [2]:
import os
import sys
import csv
from datetime import datetime

# ASEがインストールされていない場合はインストール
try:
    import ase
    from ase.io import read, write
except ImportError:
    print("ライブラリ 'ase' をインストールします...")
    !{sys.executable} -m pip install ase
    from ase.io import read, write

# --- 設定 ---
# ターゲットフォルダのパスを指定してください
target_dir = "/home/jovyan/Kaori/MD/LiB_2/structure/output/final_interfaces"
# エラーとみなすファイルサイズ（バイト単位）。2KB未満をエラーとします。
size_threshold = 2048
# 削除したファイル名を記録するCSVファイル名
log_csv_filename = "small_traj_files_log.csv"

# --- 処理記録用のリスト ---
small_files_to_log = []
cifs_created = []
trajs_deleted_after_cif = []
cifs_already_exist = []
errors_reading_traj = []

# --- メイン処理 ---
print(f"--- 処理を開始します ---")
print(f"📁 ターゲットフォルダ: {target_dir}\n")

if not os.path.isdir(target_dir):
    print(f"❌ エラー: 指定されたフォルダが見つかりません: {target_dir}")
else:
    # フォルダ内の全ファイルに対してループ処理
    for filename in sorted(os.listdir(target_dir)):
        if filename.endswith(".traj"):
            traj_filepath = os.path.join(target_dir, filename)
            
            # 1. ファイルサイズをチェック
            try:
                file_size = os.path.getsize(traj_filepath)
                if file_size < size_threshold:
                    print(f"🗑️  小さいファイルを検出: {filename} ({file_size} B) -> 削除リストに追加します。")
                    small_files_to_log.append({
                        "filename": filename,
                        "size_bytes": file_size,
                        "deleted_at": datetime.now().strftime('%Y-%m-%d %H:%M:%S')
                    })
                    os.remove(traj_filepath) # ファイルを削除
                    continue # 次のファイルへ
            except OSError as e:
                print(f"⚠️ ファイルサイズ取得/削除エラー: {filename} ({e})")
                continue

            # 2. 対応するcifファイルのパスを定義
            basename = os.path.splitext(filename)[0]
            cif_filepath = os.path.join(target_dir, f"{basename}.cif")

            # 3. cifファイルが既に存在するかチェック
            if os.path.exists(cif_filepath):
                print(f"👍 CIFは既に存在: {os.path.basename(cif_filepath)}")
                cifs_already_exist.append(os.path.basename(cif_filepath))
                continue

            # 4. cifファイルを作成し、成功したらtrajファイルを削除
            try:
                print(f"⏳ 処理中: {filename}")
                # trajectoryの最終構造(-1)を読み込む
                final_structure = read(traj_filepath, index=-1)
                
                # cifファイルとして書き出す
                write(cif_filepath, final_structure, format='cif')

                # cifファイルが正しく作成されたか確認
                if os.path.exists(cif_filepath) and os.path.getsize(cif_filepath) > 0:
                    print(f"  -> ✅ CIF作成成功: {os.path.basename(cif_filepath)}")
                    cifs_created.append(os.path.basename(cif_filepath))
                    
                    # 元のtrajファイルを削除
                    print(f"  -> 🗑️ TRAJ削除: {filename}")
                    os.remove(traj_filepath)
                    trajs_deleted_after_cif.append(filename)
                else:
                    print(f"  -> ❌ CIF作成失敗: {os.path.basename(cif_filepath)}")
                    errors_reading_traj.append(filename)

            except Exception as e:
                print(f"  -> ❌ TRAJ読み込みエラー: {filename} ({e})")
                errors_reading_traj.append(filename)

# --- ログファイルの書き出し ---
if small_files_to_log:
    log_filepath = os.path.join(target_dir, log_csv_filename)
    print(f"\n📝 小さいTRAJファイルのリストをCSVに保存します: {log_filepath}")
    try:
        with open(log_filepath, 'w', newline='', encoding='utf-8') as csvfile:
            fieldnames = ['filename', 'size_bytes', 'deleted_at']
            writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
            writer.writeheader()
            writer.writerows(small_files_to_log)
        print("  -> ✅ CSV保存完了。")
    except IOError as e:
        print(f"  -> ❌ CSV保存エラー: {e}")

print("\n--- 全ての処理が完了しました ---")

# --- 最終レポート ---
print("\n--- 処理結果サマリー ---")
print(f"📁 処理フォルダ: {target_dir}")
print("-" * 30)
print(f"🗑️ サイズが小さいため削除されたTRAJファイル: {len(small_files_to_log)} 件 (詳細は {log_csv_filename} を参照)")
print(f"✅ 新たに作成されたCIFファイル: {len(cifs_created)} 件")
print(f"🗑️ CIF作成後に削除されたTRAJファイル: {len(trajs_deleted_after_cif)} 件")
print(f"👍 既に存在したためスキップしたCIFファイル: {len(cifs_already_exist)} 件")
print(f"❌ 処理中にエラーが発生したTRAJファイル: {len(errors_reading_traj)} 件")
if errors_reading_traj:
    print("  (これらのファイルは削除されず残っています)")
    for f in errors_reading_traj: print(f"  - {f}")
print("-" * 30)

--- 処理を開始します ---
📁 ターゲットフォルダ: /home/jovyan/Kaori/MD/LiB_2/structure/output/final_interfaces

🗑️  小さいファイルを検出: Interface_Al2O3_10-12_(1, 0, 2)_on_NMC111_Co_MntoNi_40_010_(0, 1, 0)_trimmed.traj (0 B) -> 削除リストに追加します。
🗑️  小さいファイルを検出: Interface_Al2O3_10-12_(1, 0, 2)_on_NMC111_Co_MntoNi_40_012_(0, 1, 2)_trimmed.traj (0 B) -> 削除リストに追加します。
🗑️  小さいファイルを検出: Interface_Al2O3_10-12_(1, 0, 2)_on_NMC111_Co_MntoNi_40_104_(1, 0, 4)_trimmed.traj (0 B) -> 削除リストに追加します。
🗑️  小さいファイルを検出: Interface_Al2O3_10-12_(1, 0, 2)_on_NMC111_Co_decrease_001_final_trimmed.traj (0 B) -> 削除リストに追加します。
🗑️  小さいファイルを検出: Interface_Al2O3_10-12_(1, 0, 2)_on_NMC111_Co_decrease_010_final_trimmed.traj (0 B) -> 削除リストに追加します。
🗑️  小さいファイルを検出: Interface_Al2O3_10-12_(1, 0, 2)_on_NMC111_Co_decrease_104_(1, 0, 4)_optimization_final_trimmed.traj (0 B) -> 削除リストに追加します。
🗑️  小さいファイルを検出: Interface_Al2O3_10-12_(1, 0, 2)_on_NMC111_Co_decrease_final_trimmed.traj (0 B) -> 削除リストに追加します。
🗑️  小さいファイルを検出: Interface_Al2O3_10-12_(1, 0, 2)_on_NMC111_Co_increas